In [179]:
import sys
import os
import math
import torch
import torchinfo
import torch.nn as nn
import onnx
import numpy as np
from scipy.io import wavfile
from scipy.io.wavfile import write

# 1. Preprocessing

In [180]:
def prepare_training_data(config):
    in_rate, in_data = wavfile.read(config["input_audio_path"])
    out_rate, out_data = wavfile.read(config["target_audio_path"])
    
    if len(in_data) != len(out_data):
        print("input and target files have different lengths")
        sys.exit()
      
    if len(in_data.shape) > 1 or len(out_data.shape) > 1:
        print("expected mono files")
        sys.exit()

    # Convert PCM16 to FP32
    if in_data.dtype == "int16":
        in_data = in_data / 32767
        print("In data converted from PCM16 to FP32")
    if out_data.dtype == "int16":
        out_data = out_data / 32767
        print("Out data converted from PCM16 to FP32")    

    clean_data = in_data.astype(np.float32).flatten()
    target_data = out_data.astype(np.float32).flatten()

    # Split the data on a twenty percent mod
    in_train, out_train, in_val, out_val = slice_on_mod(clean_data, target_data)

    save_wav(config["output_path"] + "/train/" + config["name"] + "-input.wav", in_train)
    save_wav(config["output_path"] + "/train/" + config["name"] + "-target.wav", out_train)

    save_wav(config["output_path"] + "/test/" + config["name"] + "-input.wav", in_val)
    save_wav(config["output_path"] + "/test/" + config["name"] + "-target.wav", out_val)

    save_wav(config["output_path"] + "/val/" + config["name"] + "-input.wav", in_val)
    save_wav(config["output_path"] + "/val/" + config["name"] + "-target.wav", out_val)


In [181]:
def slice_on_mod(input_data, target_data, mod=5):
    # Split the data on a modulus.

    # Type cast to an integer the modulus
    mod = int(mod)

    # Split the data into 100 pieces
    input_split = np.array_split(input_data, 100)
    target_split = np.array_split(target_data, 100)

    val_input_data = []
    val_target_data = []
    # Traverse the range of the indexes of the input signal reversed and pop every 5th for val
    for i in reversed(range(len(input_split))):
        if i % mod == 0:
            # Store the validation data
            val_input_data.append(input_split[i])
            val_target_data.append(target_split[i])
            # Remove the validation data from training
            input_split.pop(i)
            target_split.pop(i)

    # Flatten val_data down to one dimension and concatenate
    val_input_data = np.concatenate(val_input_data)
    val_target_data = np.concatenate(val_target_data)

    # Concatenate back together
    training_input_data = np.concatenate(input_split)
    training_target_data = np.concatenate(target_split)
    return training_input_data, training_target_data, val_input_data, val_target_data

In [182]:
def save_wav(name, data):
    directory = os.path.dirname(name)

    if not os.path.exists(directory):
        os.makedirs(directory)
        
    wavfile.write(name, 44100, data.flatten().astype(np.float32))

In [183]:
importConfig = {
    "input_audio_path": "TrainingData/ts9-input.wav",
    "target_audio_path": "TrainingData/ts9-target.wav",
    "output_path": "Data",
    "name": "ts9"
}

prepare_training_data(importConfig)

# Dataloader

In [184]:
class DataSet:
    def __init__(self, data_dir='Data/'):
        self.data_dir = data_dir
        self.subsets = {}

    def create_subset(self, name, frame_len=22050):
        self.subsets[name] = {'input': None, 'target': None, 'frame_len': frame_len}

    def load_file(self, subset_name, base_filename):
        if subset_name not in self.subsets:
            raise ValueError(f"Subset '{subset_name}' does not exist")

        input_file = os.path.join(self.data_dir, f"{base_filename}-input.wav")
        target_file = os.path.join(self.data_dir, f"{base_filename}-target.wav")

        try:
            self.subsets[subset_name]['input'] = self.load_and_process(input_file, self.subsets[subset_name]['frame_len'])
            self.subsets[subset_name]['target'] = self.load_and_process(target_file, self.subsets[subset_name]['frame_len'])
        except FileNotFoundError as e:
            print(f"File Not Found: {e.filename}")
            return

    def framify(self, audio, frame_len):
        audio = np.expand_dims(audio, 1) if len(audio.shape) == 1 else audio
        seg_num = math.ceil(audio.shape[0] / frame_len)
        padded_length = seg_num * frame_len
        padded_audio = np.pad(audio, ((0, padded_length - audio.shape[0]), (0, 0)), mode='constant')
    
        reshaped_audio = np.reshape(padded_audio, (seg_num, frame_len, audio.shape[1]))
        return torch.from_numpy(reshaped_audio).float()

    def load_and_process(self, file_path, frame_len):
        sample_rate, data = wavfile.read(file_path)
        data = data.astype(np.float32)
        return self.framify(data, frame_len)

# Training

In [185]:
class StatefulLSTM(nn.Module):
    def __init__(self, input_size=1, output_size=1, hidden_size=32, skip=0, bias_fl=True):
        super(StatefulLSTM, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.skip = skip
        
        # Create dictionary of possible block types
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size)
        self.lin = nn.Linear(hidden_size, output_size, bias=bias_fl)
        self.hidden = (torch.zeros(self.input_size, 1, self.hidden_size),
                       torch.zeros(self.input_size, 1, self.hidden_size))

    # Origin forward function 
    def forward(self, x):    
        if self.skip:
            # save the residual for the skip connection
            res = x[:, :, 0:self.skip]
            x, self.hidden = self.lstm(x, self.hidden)
            return self.lin(x) + res
        else:
            x, self.hidden = self.lstm(x, self.hidden)
            x = self.lin(x)
            return x

    # detach hidden state, this resets gradient tracking on the hidden state
    def detach_hidden(self):
        if self.hidden.__class__ == tuple:
            self.hidden = tuple([h.clone().detach() for h in self.hidden])
        else:
            self.hidden = self.hidden.clone().detach()

    def reset_hidden(self, batch_size):
        self.hidden = (torch.zeros(self.input_size, batch_size, self.hidden_size), 
                       torch.zeros(self.input_size, batch_size, self.hidden_size))

    # train_epoch runs one epoch of training
    def train_epoch(self, input_data, target_data, loss_fcn, optim, bs, init_len=200, up_fr=1000):

        # shuffle the segments at the start of the epoch
        shuffle = torch.randperm(input_data.shape[1])
    
        print(f"self.reset_hidden(bs): {bs}")
        self.reset_hidden(bs)
        
        ep_loss = 0
        #Iterate over batches of {bs} batches
        for batch_i in range(math.ceil(shuffle.shape[0] / bs)):
            # Load batch of shuffled segments
            input_batch = input_data[:, shuffle[batch_i * bs:(batch_i + 1) * bs], :]
            target_batch = target_data[:, shuffle[batch_i * bs:(batch_i + 1) * bs], :]

            # Initialise network hidden state by processing some samples then zero the gradient buffers
            # For training processing eine Anfangssequenz, damit ein brauchbarer hidden state vorliegt
            # Training startet erst nach! einem eingelaufen hidden state
            print(input_batch[0:init_len, :, :].shape)
            out = self(input_batch[0:init_len, :, :])
                        
            self.zero_grad()

            # Choose the starting index for processing the rest of the batch sequence, in chunks of args.up_fr
            start_i = init_len
            batch_loss = 0
            # Iterate over the remaining samples in the mini batch
            for k in range(math.ceil((input_batch.shape[0] - init_len) / up_fr)):
                # Process input batch with neural network                
                output = self(input_batch[start_i:start_i + up_fr, :, :])

                # Calculate loss and update network parameters
                loss = loss_fcn(output, target_batch[start_i:start_i + up_fr, :, :])
                loss.backward()
                optim.step()

                print(f"loss: {loss}")

                # Set the network hidden state, to detach it from the computation graph
                # Die Gradienteninformation die einhergeht mit dem hidden state ist connected zu dem
                # Computiaonal Graf von den vorherigen outputs. Da wir immer nur den letzten hidden state
                # zur berechnung brauchen und den state auch nicht updaten wollen - wird er hier detached
                # detached = wir löchen die Gradienteninformation 
                # https://discuss.pytorch.org/t/stupid-question-why-do-you-have-to-detach-the-hidden-state-of-lstms-but-not-the-hidden-state-of-a-linear-network/95089/3                
                self.detach_hidden()
                self.zero_grad()

                # Update the start index for the next iteration and add the loss to the batch_loss total
                start_i += up_fr
                batch_loss += loss

            # Add the average batch loss to the epoch loss and reset the hidden states to zeros
            ep_loss += batch_loss / (k + 1)
        
        return ep_loss / (batch_i + 1)

In [186]:
class ESRLoss(nn.Module):
    def __init__(self):
        super(ESRLoss, self).__init__()
        self.epsilon = 0.00001

    def forward(self, output, target):
        loss = torch.add(target, -output)
        loss = torch.pow(loss, 2)
        loss = torch.mean(loss)
        energy = torch.mean(torch.pow(target, 2)) + self.epsilon
        loss = torch.div(loss, energy)
        return loss
class DCLoss(nn.Module):
    def __init__(self):
        super(DCLoss, self).__init__()
        self.epsilon = 0.00001

    def forward(self, output, target):
        loss = torch.pow(torch.add(torch.mean(target, 0), -torch.mean(output, 0)), 2)
        loss = torch.mean(loss)
        energy = torch.mean(torch.pow(target, 2)) + self.epsilon
        loss = torch.div(loss, energy)
        return loss      
class LossWrapper(nn.Module):
    def __init__(self, losses):
        super(LossWrapper, self).__init__()
        loss_dict = {'ESR': ESRLoss(), 'DC': DCLoss()}

        loss_functions = [[loss_dict[key], value] for key, value in losses.items()]

        self.loss_functions = tuple([items[0] for items in loss_functions])
        try:
            self.loss_factors = tuple(torch.Tensor([items[1] for items in loss_functions]))
        except IndexError:
            self.loss_factors = torch.ones(len(self.loss_functions))

    def forward(self, output, target):
        loss = 0
        for i, losses in enumerate(self.loss_functions):
            loss += torch.mul(losses(output, target), self.loss_factors[i])
        return loss

In [187]:
config = {
    "input_size": 1, # Number of channels
    "output_size": 1, # Number of channels
    "skip_con": 0, # is there a skip connection for the input to the output
    "epochs": 100,
    "batch_size": 16,
    "init_length": 200, # Number of sequence samples to process before starting weight updates
    "up_fr": 1000, # For recurrent models, number of samples to run in between updating network weights
    "learning_rate": 0.0005, 
    "hidden_size": 32,
    "loss_fcns": {"ESR": 0.75, "DC": 0.25},
    "hardware_device": "ts9",
    "save_location": "Results-PyTorch"
}

In [188]:
if torch.cuda.is_available():
    print('CUDA device available')
    torch.set_default_dtype(torch.cuda.FloatTensor)
    torch.cuda.set_device(0)
else:
    print('CUDA device not available')

print("Creating Stateful LSTM")
network = StatefulLSTM(input_size=config["input_size"], 
                       output_size=config["output_size"], 
                       hidden_size=config["hidden_size"], 
                       skip=config["skip_con"])

optimiser = torch.optim.Adam(network.parameters(), lr=config["learning_rate"], weight_decay=1e-4)
#scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimiser, 'min', factor=0.5, patience=5, verbose=True)
#loss_functions = torch.nn.MSELoss(reduction='sum')
loss_functions = LossWrapper(config["loss_fcns"])

summary = torchinfo.summary(network, (1, 1, 1), device=torch.device("cpu"))
print(summary)

dataset = DataSet()
dataset.create_subset('train', frame_len=22050)
dataset.load_file('train', os.path.join('train', config["hardware_device"]))

dataset.create_subset('val')
dataset.load_file('val', os.path.join('val', config["hardware_device"]))

CUDA device not available
Creating Stateful LSTM
Layer (type:depth-idx)                   Output Shape              Param #
StatefulLSTM                             [1, 1, 1]                 --
├─LSTM: 1-1                              [1, 1, 32]                4,480
├─Linear: 1-2                            [1, 1, 1]                 33
Total params: 4,513
Trainable params: 4,513
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.02
Estimated Total Size (MB): 0.02


In [ ]:
for epoch in range(1, config["epochs"] + 1):
    print("Epoch: ", epoch)

    epoch_loss = network.train_epoch(dataset.subsets['train']['input'],
                                     dataset.subsets['train']['target'],
                                     loss_functions,
                                     optimiser,
                                     config['batch_size'],
                                     config['init_length'],
                                     config['up_fr'])

    print("Epoch loss:", epoch_loss)

In [ ]:
#network.reset_hidden(1)
torch.save(network.state_dict(), "models/ht1.pth")

In [189]:
model = StatefulLSTM(input_size=config["input_size"], 
                           output_size=config["output_size"], 
                           hidden_size=config["hidden_size"], 
                           skip=config["skip_con"])

model.load_state_dict(torch.load("models/ht1.pth", map_location=torch.device('cpu')))

<All keys matched successfully>

In [190]:
input = dataset.subsets['train']['input'].swapaxes(0, 1)
print(input.shape)

torch.Size([22050, 299, 1])


In [191]:
output = torch.zeros_like(input)
model.reset_hidden(1)
with (torch.no_grad()):
    for i in range(0, input.shape[1]):
        output[:, i:i+1, :] = model(input[:, i:i+1, :])

In [192]:
output_concat = output[:, 0, :]
input_concat = input[:, 0, :]
for i in range(1, output.shape[1]):
    output_concat = torch.cat((output_concat, output[:, i, :]), 0)
    input_concat = torch.cat((input_concat, input[:, i, :]), 0)

In [193]:
write(os.path.join("input-pytorch.wav"), 44100, input_concat.cpu().numpy().reshape(-1, 1))
write(os.path.join("train_out-pytorch.wav"), 44100, output_concat.cpu().numpy().reshape(-1, 1))

In [194]:
name = "model_0"
model.reset_hidden(1)
model_scripted = torch.jit.script(model)
model_scripted.save("models/"+name+"/"+"stateful-lstm.pt")

In [195]:
example_input = torch.rand(64, 1, 1)
filepath = "models/"+name+"/"+"stateful-lstm-libtorch.onnx"
model.reset_hidden(1)

# Export the model
torch.onnx.export(model=model,
                args=example_input,
                f=filepath,
                export_params=True,
                opset_version=13,
                do_constant_folding=True,
                input_names = ['input'],
                output_names = ['output'])

/home/faressc/msc-audiocommunation/master-thesis/stateful-lstm/venv/lib/python3.11/site-packages/torch/onnx/symbolic_opset9.py:4662: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(


In [196]:
loaded_model = torch.jit.load("models/"+name+"/"+"stateful-lstm.pt")

# Example input
sequence_length = 100  # adjust as per your model's training
batch_size = 1         # can be set to 1 for testing individual sequences
input_size = loaded_model.input_size  # should match the model's expected input size\
print(input_size)

test_input = torch.zeros(64, 1, 1)
test_output = loaded_model(test_input)
print(test_output)
test_output = loaded_model(test_input)
print(test_output)

1
tensor([[[-1.5788e-02]],

        [[-1.8019e-02]],

        [[-2.0228e-02]],

        [[-2.2176e-02]],

        [[-2.3751e-02]],

        [[-2.4923e-02]],

        [[-2.5717e-02]],

        [[-2.6187e-02]],

        [[-2.6396e-02]],

        [[-2.6399e-02]],

        [[-2.6247e-02]],

        [[-2.5975e-02]],

        [[-2.5612e-02]],

        [[-2.5178e-02]],

        [[-2.4686e-02]],

        [[-2.4149e-02]],

        [[-2.3572e-02]],

        [[-2.2962e-02]],

        [[-2.2324e-02]],

        [[-2.1661e-02]],

        [[-2.0977e-02]],

        [[-2.0273e-02]],

        [[-1.9554e-02]],

        [[-1.8820e-02]],

        [[-1.8075e-02]],

        [[-1.7320e-02]],

        [[-1.6557e-02]],

        [[-1.5789e-02]],

        [[-1.5017e-02]],

        [[-1.4243e-02]],

        [[-1.3470e-02]],

        [[-1.2698e-02]],

        [[-1.1930e-02]],

        [[-1.1168e-02]],

        [[-1.0412e-02]],

        [[-9.6644e-03]],

        [[-8.9272e-03]],

        [[-8.2015e-03]],

        [[

In [197]:
test_input = torch.zeros(64, 1, 1)
test_output = loaded_model(test_input)
print(test_output)
test_output = loaded_model(test_input)
print(test_output)

tensor([[[-0.0135]],

        [[-0.0136]],

        [[-0.0136]],

        [[-0.0136]],

        [[-0.0137]],

        [[-0.0136]],

        [[-0.0136]],

        [[-0.0136]],

        [[-0.0135]],

        [[-0.0135]],

        [[-0.0134]],

        [[-0.0133]],

        [[-0.0132]],

        [[-0.0131]],

        [[-0.0130]],

        [[-0.0128]],

        [[-0.0127]],

        [[-0.0125]],

        [[-0.0124]],

        [[-0.0122]],

        [[-0.0120]],

        [[-0.0118]],

        [[-0.0116]],

        [[-0.0114]],

        [[-0.0112]],

        [[-0.0110]],

        [[-0.0108]],

        [[-0.0106]],

        [[-0.0104]],

        [[-0.0102]],

        [[-0.0099]],

        [[-0.0097]],

        [[-0.0095]],

        [[-0.0093]],

        [[-0.0091]],

        [[-0.0088]],

        [[-0.0086]],

        [[-0.0084]],

        [[-0.0082]],

        [[-0.0080]],

        [[-0.0078]],

        [[-0.0076]],

        [[-0.0074]],

        [[-0.0072]],

        [[-0.0070]],

        [[

In [200]:
import onnxruntime
import numpy as np

# Load the ONNX model
onnx_model_path = "models/"+name+"/"+"stateful-lstm-libtorch.onnx"
ort_session = onnxruntime.InferenceSession(onnx_model_path)

# Create an input tensor with shape (1, 1, 1) filled with zeros
test_input = np.zeros((64, 1, 1), dtype=np.float32)

# Run inference on the ONNX model
ort_inputs = {"input": test_input}
ort_outputs = ort_session.run(None, ort_inputs)
ort_outputs2 = ort_session.run(None, ort_inputs)

# Print the output
print(ort_outputs)
# Print the output
print(ort_outputs2)


[array([[[-1.5788125e-02]],

       [[-1.8018806e-02]],

       [[-2.0228103e-02]],

       [[-2.2175824e-02]],

       [[-2.3750748e-02]],

       [[-2.4922887e-02]],

       [[-2.5717199e-02]],

       [[-2.6187457e-02]],

       [[-2.6395619e-02]],

       [[-2.6399426e-02]],

       [[-2.6246896e-02]],

       [[-2.5975216e-02]],

       [[-2.5612168e-02]],

       [[-2.5177766e-02]],

       [[-2.4686458e-02]],

       [[-2.4148669e-02]],

       [[-2.3572080e-02]],

       [[-2.2962425e-02]],

       [[-2.2324301e-02]],

       [[-2.1661377e-02]],

       [[-2.0976823e-02]],

       [[-2.0273432e-02]],

       [[-1.9553736e-02]],

       [[-1.8820049e-02]],

       [[-1.8074643e-02]],

       [[-1.7319603e-02]],

       [[-1.6557012e-02]],

       [[-1.5788857e-02]],

       [[-1.5017055e-02]],

       [[-1.4243471e-02]],

       [[-1.3469983e-02]],

       [[-1.2698361e-02]],

       [[-1.1930340e-02]],

       [[-1.1167594e-02]],

       [[-1.0411795e-02]],

       [[-9.6645029